In [1]:
import pandas as pd
import numpy as np 

In [3]:
file_path = 'valeursfoncieres-2023.txt'
data = pd.read_csv(file_path, delimiter='|', parse_dates=['Date mutation'])

C:\Users\moham\AppData\Local\Temp\ipykernel_51892\1853108269.py:2: DtypeWarning: Columns (18,23,24,26,28,29,31,32,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, delimiter='|', parse_dates=['Date mutation'])


In [7]:
def nettoyer_donnees(df):
    """
    Effectuer un nettoyage complet des données du dataframe.
    """
    
    # Supprimer les doublons
    df.drop_duplicates(inplace=True)
    
    # Convertir 'Valeur fonciere' et des colonnes similaires en numérique, en traitant les séparateurs décimaux
    for col in df.select_dtypes(include='object').columns:
        if df[col].str.contains(',', regex=False).any():
            df[col] = pd.to_numeric(df[col].str.replace(',', '.'), errors='coerce')
    
    # Supprimer les colonnes ayant plus de 90% de valeurs manquantes
    seuil_na = 0.9
    min_non_na = df.shape[0] * (1 - seuil_na)
    df.dropna(axis=1, thresh=min_non_na, inplace=True)
    
    # Remplir les valeurs numériques manquantes avec la médiane et les catégorielles avec la mode
    for col in df.select_dtypes(include=np.number).columns:
        df[col].fillna(df[col].median(), inplace=True)
    
    for col in df.select_dtypes(include='object').columns:
        df[col].fillna(df[col].mode()[0], inplace=True)
    
    # Traitement des valeurs aberrantes dans les colonnes numériques
    for col in df.select_dtypes(include=np.number).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        limite_inf = Q1 - 1.5 * IQR
        limite_sup = Q3 + 1.5 * IQR
        
        df[col] = np.where(df[col] < limite_inf, limite_inf, df[col])
        df[col] = np.where(df[col] > limite_sup, limite_sup, df[col])
    
    # D'autres contrôles de formatage de données et d'intégrité peuvent être ajoutés ici
    
    return df

def calculer_seuil_colonne(df):
    # Calculer dynamiquement un seuil pour la duplication des colonnes
    taux_duplication = df.apply(lambda col: col.duplicated().mean())
    Q3 = taux_duplication.quantile(0.75)
    IQR = Q3 - taux_duplication.quantile(0.25)
    return Q3 + 1.5 * IQR

def identifier_colonnes_similaires(df, seuil_duplication):
    # Identifier les colonnes pour vérifier la similarité en fonction du seuil de duplication
    return [col for col in df.columns if df[col].duplicated().mean() > seuil_duplication]

def calculer_seuil_ligne(nb_colonnes_similaires):
    # Calculer dynamiquement un seuil pour la similarité des lignes
    return nb_colonnes_similaires - 1

def supprimer_lignes_similaires(df, colonnes_subset, seuil_similarite):
    """
    Supprimer les lignes qui ont un haut degré de similarité à travers le subset de colonnes spécifié.
    """
    # Créer un tuple de valeurs pour chaque ligne basé sur le subset de colonnes
    df['combine'] = df[colonnes_subset].apply(lambda row: tuple(row), axis=1)
    
    # Trouver les tuples dupliqués basés sur le seuil de similarité
    dupliques = df['combine'].duplicated(keep=False)
    groupes = df[dupliques].groupby('combine').filter(lambda x: len(x) > seuil_similarite).index
    
    # Supprimer les lignes identifiées et enlever la colonne temporaire 'combine'
    df_nettoye = df.drop(index=groupes).drop(columns=['combine'])
    return df_nettoye

# Commencer le pipeline de nettoyage
donnees = nettoyer_donnees(data)

# Calculer le seuil de duplication basé sur les données
seuil_duplication_colonne = calculer_seuil_colonne(data)

# Identifier les colonnes avec des taux élevés de duplication
colonnes_similaires = identifier_colonnes_similaires(data, seuil_duplication_colonne)

# Si il n'y a pas assez de colonnes similaires, nous pouvons décider de ne pas supprimer de lignes
if len(colonnes_similaires) > 1:
    # Calculer le seuil de similarité des lignes basé sur le nombre de colonnes similaires
    seuil_similarite_ligne = calculer_seuil_ligne(len(colonnes_similaires))

    # Supprimer les lignes qui sont similaires basé sur le seuil calculé
    donnees = supprimer_lignes_similaires(data, colonnes_similaires, seuil_similarite_ligne)


In [8]:
data.head()

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Code postal,Commune,Code departement,...,1er lot,Surface Carrez du 1er lot,2eme lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
0,1.0,05/01/2023,Vente,644800.0,67.0,ALL,0124,1630.0,ST-GENIS-POUILLY,1,...,29,54.1,6.0,1.0,3.0,Dépendance,0.0,0.0,S,610.0
1,1.0,05/01/2023,Vente,644800.0,67.0,ALL,0124,1630.0,ST-GENIS-POUILLY,1,...,8,54.1,6.0,1.0,3.0,Dépendance,0.0,0.0,S,610.0
2,1.0,05/01/2023,Vente,644800.0,67.0,ALL,0124,1630.0,ST-GENIS-POUILLY,1,...,22,54.1,6.0,1.0,2.0,Appartement,86.0,5.0,S,610.0
3,1.0,03/01/2023,Vente,152200.0,67.0,RTE,0107,1450.0,SERRIERES-SUR-AIN,1,...,1.0,54.1,6.0,0.0,1.0,Maison,64.0,3.0,S,988.0
4,1.0,05/01/2023,Vente,269000.0,67.0,CHE,0040,1800.0,SAINT-JEAN-DE-NIOST,1,...,1.0,54.1,6.0,0.0,3.0,Dépendance,0.0,0.0,S,835.0
